In [77]:
source="java"
target="cs"
lr=1e-4
batch_size=64
beam_size=10
source_length=320
target_length=256
output_dir=f"saved_models/{source}-{target}/"
train_file=f"data/train.java-cs.txt.{source},data/train.java-cs.txt.{target}"
dev_file=f"data/valid.java-cs.txt.{source},data/valid.java-cs.txt.{target}"
epochs=2
pretrained_model="microsoft/graphcodebert-base"

In [78]:
from __future__ import absolute_import
import os
import sys
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from bleu import _bleu
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaModel, RobertaTokenizer)
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaModel, RobertaTokenizer)}

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
from parser import DFG_python,DFG_java,DFG_ruby,DFG_go,DFG_php,DFG_javascript,DFG_csharp
from parser import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index)
from tree_sitter import Language, Parser

In [79]:
dfg_function={
    'python':DFG_python,
    'java':DFG_java,
    'ruby':DFG_ruby,
    'go':DFG_go,
    'php':DFG_php,
    'javascript':DFG_javascript,
    'c_sharp':DFG_csharp,
}

logger = logging.getLogger(__name__)
#load parsers
parsers={}        
for lang in dfg_function:
    # print(Language)
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]    
    parsers[lang]= parser

/opt/conda/envs/pytorch/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


### Helper functions

In [80]:
class Example(object):
    """A single training/test example."""
    def __init__(self,
                 source,
                 target,
                 lang
                 ):
        self.source = source
        self.target = target
        self.lang=lang

In [81]:
def extract_dataflow(code, parser,lang):
    #remove comments
    try:
        code=remove_comments_and_docstrings(code,lang)
    except:
        pass    
    #obtain dataflow
    if lang=="php":
        code="<?php"+code+"?>"    
    try:
        tree = parser[0].parse(bytes(code,'utf8'))    
        root_node = tree.root_node  
        tokens_index=tree_to_token_index(root_node)     
        code=code.split('\n')
        code_tokens=[index_to_code_token(x,code) for x in tokens_index]  
        index_to_code={}
        for idx,(index,code) in enumerate(zip(tokens_index,code_tokens)):
            index_to_code[index]=(idx,code)  
        try:
            DFG,_=parser[1](root_node,index_to_code,{}) 
        except:
            DFG=[]
        DFG=sorted(DFG,key=lambda x:x[1])
        indexs=set()
        for d in DFG:
            if len(d[-1])!=0:
                indexs.add(d[1])
            for x in d[-1]:
                indexs.add(x)
        new_DFG=[]
        for d in DFG:
            if d[1] in indexs:
                new_DFG.append(d)
        dfg=new_DFG
    except:
        dfg=[]
    return code_tokens,dfg

In [82]:
def read_examples(filename):
    """Read examples from filename."""
    examples=[]
    source,target=filename.split(',')
    lang='java'
    if source[-1]=='s':
        lang='c_sharp'
        
    with open(source,encoding="utf-8") as f1,open(target,encoding="utf-8") as f2:
        for line1,line2 in zip(f1,f2):
            line1=line1.strip()
            line2=line2.strip()
            examples.append(
                Example(
                    source=line1,
                    target=line2,
                    lang=lang
                        ) 
            )

    return examples

**Define the input for BERT model**

In [83]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,                 
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.position_idx = position_idx
        self.dfg_to_code = dfg_to_code
        self.dfg_to_dfg = dfg_to_dfg
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask       
        

In [84]:
def convert_examples_to_features(examples, tokenizer, args,stage=None):
    features = []
    for example_index, example in enumerate(tqdm(examples,total=len(examples))):
        ##extract data flow
        code_tokens,dfg=extract_dataflow(example.source,
                                         parsers["c_sharp" if args.source_lang == "cs" else "java"],
                                         "c_sharp" if args.source_lang == "cs" else "java")
        code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]
        ori2cur_pos={}
        ori2cur_pos[-1]=(0,0)
        for i in range(len(code_tokens)):
            ori2cur_pos[i]=(ori2cur_pos[i-1][1],ori2cur_pos[i-1][1]+len(code_tokens[i]))    
        code_tokens=[y for x in code_tokens for y in x]  
        
        #truncating
        code_tokens=code_tokens[:args.max_source_length-3][:512-3]
        source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
        position_idx = [i+tokenizer.pad_token_id + 1 for i in range(len(source_tokens))]
        dfg=dfg[:args.max_source_length-len(source_tokens)]
        source_tokens+=[x[0] for x in dfg]
        position_idx+=[0 for x in dfg]
        source_ids+=[tokenizer.unk_token_id for x in dfg]
        padding_length=args.max_source_length-len(source_ids)
        position_idx+=[tokenizer.pad_token_id]*padding_length
        source_ids+=[tokenizer.pad_token_id]*padding_length      
        source_mask = [1] * (len(source_tokens))
        source_mask+=[0]*padding_length        
        
        #reindex
        reverse_index={}
        for idx,x in enumerate(dfg):
            reverse_index[x[1]]=idx
        for idx,x in enumerate(dfg):
            dfg[idx]=x[:-1]+([reverse_index[i] for i in x[-1] if i in reverse_index],)    
        dfg_to_dfg=[x[-1] for x in dfg]
        dfg_to_code=[ori2cur_pos[x[1]] for x in dfg]
        length=len([tokenizer.cls_token])
        dfg_to_code=[(x[0]+length,x[1]+length) for x in dfg_to_code]        
      

        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example.target)[:args.max_target_length-2]
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]            
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = args.max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length   
   
        if example_index < 5:
            if stage=='train':
                logger.info("*** Example ***")
                logger.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                logger.info("source_ids: {}".format(' '.join(map(str, source_ids))))
                logger.info("source_mask: {}".format(' '.join(map(str, source_mask))))
                logger.info("position_idx: {}".format(position_idx))
                logger.info("dfg_to_code: {}".format(' '.join(map(str, dfg_to_code))))
                logger.info("dfg_to_dfg: {}".format(' '.join(map(str, dfg_to_dfg))))
                
                logger.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                logger.info("target_ids: {}".format(' '.join(map(str, target_ids))))
                logger.info("target_mask: {}".format(' '.join(map(str, target_mask))))
       
        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features

### Prepare fine-tuning parameters

In [85]:
import argparse

# Simulate argparse for Jupyter
class Args:
    model_type = "roberta"
    model_name_or_path = "roberta-base"
    output_dir = "./output"
    load_model_path = None
    train_filename = None
    dev_filename = None
    test_filename = None
    source_lang = "en"
    config_name = ""
    tokenizer_name = ""
    max_source_length = 64
    max_target_length = 32
    do_train = True
    do_eval = True
    do_test = False
    do_lower_case = False
    no_cuda = False
    train_batch_size = 256
    eval_batch_size = 512
    gradient_accumulation_steps = 1
    learning_rate = 5e-5
    beam_size = 10
    weight_decay = 0.0
    adam_epsilon = 1e-8
    max_grad_norm = 1.0
    num_train_epochs = 3
    max_steps = -1
    eval_steps = -1
    train_steps = -1
    warmup_steps = 0
    local_rank = -1
    seed = 42

# Create an instance of Args
args = Args()

# Print the arguments for verification
print(f"Arguments: {args}")
args.model_type = "roberta"
args.source_lang = source
args.target_lang = target
args.model_name_or_path = pretrained_model
args.tokenizer_name = "microsoft/graphcodebert-base"
args.config_name = "microsoft/graphcodebert-base"
args.train_filename = train_file
args.dev_filename = dev_file
args.output_dir = output_dir
args.learning_rate = lr
args.num_train_epochs = epochs
args.train_batch_size = batch_size
args.eval_batch_size = batch_size
args.max_source_length = source_length
args.max_target_length = target_length

Arguments: <__main__.Args object at 0x7fc0e7450910>


In [86]:
# prepare parsers for each language
parsers={}        
for lang in dfg_function:
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]    
    parsers[lang]= parser

In [87]:
def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYHTONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(args.seed)

Define dataset

In [88]:
class TextDataset(Dataset):
    def __init__(self, examples, args):
        self.examples = examples
        self.args=args  
        
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, item):
        #calculate graph-guided masked function
        attn_mask=np.zeros((self.args.max_source_length,self.args.max_source_length),dtype=np.bool)
        #calculate begin index of node and max length of input
        node_index=sum([i>1 for i in self.examples[item].position_idx])
        max_length=sum([i!=1 for i in self.examples[item].position_idx])
        #sequence can attend to sequence
        attn_mask[:node_index,:node_index]=True
        #special tokens attend to all tokens
        for idx,i in enumerate(self.examples[item].source_ids):
            if i in [0,2]:
                attn_mask[idx,:max_length]=True
        #nodes attend to code tokens that are identified from
        for idx,(a,b) in enumerate(self.examples[item].dfg_to_code):
            if a<node_index and b<node_index:
                attn_mask[idx+node_index,a:b]=True
                attn_mask[a:b,idx+node_index]=True
        #nodes attend to adjacent nodes         
        for idx,nodes in enumerate(self.examples[item].dfg_to_dfg):
            for a in nodes:
                if a+node_index<len(self.examples[item].position_idx):
                    attn_mask[idx+node_index,a+node_index]=True  
                    
        return (torch.tensor(self.examples[item].source_ids),
                torch.tensor(self.examples[item].source_mask),
                torch.tensor(self.examples[item].position_idx),
                torch.tensor(attn_mask), 
                torch.tensor(self.examples[item].target_ids),
                torch.tensor(self.examples[item].target_mask),)
    


In [89]:
# Setup CUDA, GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()
args.device = device

In [90]:
# make dir if output_dir not exist
if os.path.exists(args.output_dir) is False:
    os.makedirs(args.output_dir)

**Training helpers**

In [94]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name)
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name )

In [95]:
def save_model(model, output_dir, step):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model_to_save = model.module if hasattr(model, 'module') else model 
    output_model_file = os.path.join(output_dir, "model.{}.bin".format(step))
    torch.save(model_to_save.state_dict(), output_model_file)

In [96]:
def calculate_bleu(model, dev_dataset):
    if 'dev_bleu' in dev_dataset:
        eval_examples,eval_data=dev_dataset['dev_bleu']
    else:
        eval_examples = read_examples(args.dev_filename)
        eval_examples = random.sample(eval_examples,min(1000,len(eval_examples)))
        eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='test')
        eval_data = TextDataset(eval_features,args)
        dev_dataset['dev_bleu']=eval_examples,eval_data
        
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=4)
    model.eval() 
    p=[]
    for batch in eval_dataloader:
        batch = tuple(t.to(device) for t in batch)
        source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch                 
        with torch.no_grad():
            preds = model(source_ids,source_mask,position_idx,att_mask)  
            for pred in preds:
                t=pred[0].cpu().numpy()
                t=list(t)
                if 0 in t:
                    t=t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                p.append(text)
    model.train()
    predictions=[]
    accs = []
    with open(os.path.join(args.output_dir,"dev.output"),'w') as f, open(os.path.join(args.output_dir,"dev.gold"),'w') as f1:
        for ref,gold in zip(p,eval_examples):
            predictions.append(ref)
            f.write(ref+'\n')
            f1.write(gold.target+'\n')     
            accs.append(ref==gold.target)

    dev_bleu=round(_bleu(os.path.join(args.output_dir, "dev.gold"), os.path.join(args.output_dir, "dev.output")),2)
    xmatch=round(np.mean(accs)*100,4)
    return dev_bleu,xmatch
    

In [97]:
def calculate_bleu(model, eval_examples, eval_dataloader):
    
    model.eval() 
    p=[]
    for batch in eval_dataloader:
        batch = tuple(t.to(device) for t in batch)
        source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch                 
        with torch.no_grad():
            preds = model(source_ids, source_mask, position_idx, att_mask, target_ids, target_mask) 
            for pred in preds:
                t=pred[0].cpu().numpy()
                t=list(t)
                if 0 in t:
                    t=t[:t.index(0)]
                text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
                p.append(text)
    model.train()
    predictions=[]
    accs = []
    with open(os.path.join(args.output_dir,"dev.output"),'w') as f, open(os.path.join(args.output_dir,"dev.gold"),'w') as f1:
        for ref,gold in zip(p, eval_examples):
            predictions.append(ref)
            f.write(ref+'\n')
            f1.write(gold.target+'\n')     
            accs.append(ref==gold.target)

    dev_bleu=round(_bleu(os.path.join(args.output_dir, "dev.gold"), os.path.join(args.output_dir, "dev.output")),2)
    xmatch=round(np.mean(accs)*100,4)
    
    return dev_bleu,xmatch

In [20]:
# # push to huggingface
# !huggingface-cli login
# repo_url = "https://huggingface.co/judynguyen16/graphcodebert--code-translation-java-cs"

## Training with Lightning Pytorch

### Prepare the dataloaders

In [17]:
# # ---------*****---------
# # Prepare training loader and fine-tuning

# config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
# config = config_class.from_pretrained(args.config_name)
# tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name )

# train_examples = read_examples(args.train_filename)
# train_features = convert_examples_to_features(train_examples, tokenizer, args,stage='train')
# train_data = TextDataset(train_features,args)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size//args.gradient_accumulation_steps,num_workers=54)
# num_train_optimization_steps =  args.train_steps

# #Start training
# logger.info("***** Running training *****")
# logger.info("  Num examples = %d", len(train_examples))
# logger.info("  Batch size = %d", args.train_batch_size)
# logger.info("  Num epoch = %d", args.num_train_epochs)

100%|██████████| 10300/10300 [00:22<00:00, 457.98it/s]
/opt/conda/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 54 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
11/26/2024 17:49:52 - INFO - __main__ -   ***** Running training *****
11/26/2024 17:49:52 - INFO - __main__ -     Num examples = 10300
11/26/2024 17:49:52 - INFO - __main__ -     Batch size = 64
11/26/2024 17:49:52 - INFO - __main__ -     Num epoch = 2


In [98]:
# validation loader
dev_dataset={}
if 'dev_loss' in dev_dataset:
    eval_examples,eval_data=dev_dataset['dev_loss']
else:
    eval_examples = read_examples(args.dev_filename)
    eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='dev')
    eval_data = TextDataset(eval_features,args)
    dev_dataset['dev_loss']=eval_examples,eval_data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=54)

logger.info("\n***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
logger.info("  Batch size = %d", args.eval_batch_size)

100%|██████████| 500/500 [00:01<00:00, 399.43it/s]
11/26/2024 21:00:58 - INFO - __main__ -   
***** Running evaluation *****
11/26/2024 21:00:58 - INFO - __main__ -     Num examples = 500
11/26/2024 21:00:58 - INFO - __main__ -     Batch size = 64


In [101]:
# test loader
files=[]
if args.dev_filename is not None:
    files.append(args.dev_filename)
if args.test_filename is not None:
    files.append(args.test_filename)
    
for idx,file in enumerate(files):   
    logger.info("Test file: {}".format(file))
    eval_examples = read_examples(file)
    eval_examples = eval_examples[:50]
    eval_features = convert_examples_to_features(eval_examples, tokenizer, args,stage='test')
    eval_data = TextDataset(eval_features,args) 

    # Calculate bleu
    eval_sampler = SequentialSampler(eval_data)
    test_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=args.eval_batch_size,num_workers=16)

11/26/2024 21:02:00 - INFO - __main__ -   Test file: data/valid.java-cs.txt.java,data/valid.java-cs.txt.cs
100%|██████████| 50/50 [00:00<00:00, 561.89it/s]


### Define the LightningModule model for training

In [103]:
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name)
tokenizer = tokenizer_class.from_pretrained(args.tokenizer_name )

#budild model
encoder = model_class.from_pretrained(args.model_name_or_path,config=config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model=Seq2Seq(encoder=encoder,decoder=decoder,config=config,
                beam_size=args.beam_size,max_length=args.max_target_length,
                sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)

# if args.load_model_path is not None:
pretrained_model_path = "saved_models/java-cs/checkpoint-best-ppl/pytorch_model.bin"
print("reload model from {}".format(pretrained_model_path))
model.load_state_dict(torch.load(pretrained_model_path))
    
model.to(device)

/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


reload model from saved_models/java-cs/checkpoint-best-ppl/pytorch_model.bin


Seq2Seq(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

### Evaluation Starts

In [71]:
# # evaluate the model
# np.bool = np.bool_
# model.eval()
# dev_bleu,xmatch=calculate_bleu(model, eval_examples, test_dataloader)
# print(f"BLEU: {dev_bleu}, X-Match: {xmatch}")

In [110]:
model.eval() 
p=[]
for batch in tqdm(test_dataloader,total=len(test_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch                    
    with torch.no_grad():
        preds = model(source_ids,source_mask,position_idx,att_mask, None, None)  
        for pred in preds:
            t=pred[0].cpu().numpy()
            t=list(t)
            if 0 in t:
                t=t[:t.index(0)]
            text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
            p.append(text)
# model.train()

100%|██████████| 1/1 [00:43<00:00, 43.94s/it]


In [111]:
predictions=[]
accs = []
with open(os.path.join(args.output_dir,"dev.output"),'w') as f, open(os.path.join(args.output_dir,"dev.gold"),'w') as f1:
    for ref, gold in zip(p, eval_examples):
        predictions.append(ref)
        f.write(ref+'\n')
        f1.write(gold.target+'\n')     
        accs.append(ref==gold.target)

In [112]:
predictions

['public DVRecord(RecordInputStream in1){_option_flags = in1.ReadInt();_promptTitle = in1.ReadInt();_errorTitle = in1.ReadUnicodeString(in1);_errorTitle = in1.ReadUnicodeString(in1);int field_1 = in1.ReadUShort();_size_1 = in1.ReadShort();_size_1 = in1.ReadShort();_not_used_formula = in1.ReadShort();_formula1 = NPOI.Read(field_size_first_formula, in1.ReadShort();_formula2 = Formula.Read(field_size_size_formula, in1);_regions = Formula.Read(in1);}',
 'public override string ToString(){return pattern();}',
 'public InsertInstanceRequest(): base("Ots", "2016-06-20", "InsertInstance", "ots", "openAPI"){Method = MethodType.POST;}',
 'public override bool contains(object o){return indexOf(o) != -1;}',
 'public java.nio.ByteBuffer encode(string s){return encode(java.nio.CharBuffer.wrap(s));}',
 'public override bool RequiresCommitBody(){return false;}',
 'public virtual string GetKey(){return RawParseUtils.Decode(enc, buffer, keyStart, keyEnd);}',
 'public override ValueEval Evaluate(int srcR

In [113]:
dev_bleu=round(_bleu(os.path.join(args.output_dir, "dev.gold"), os.path.join(args.output_dir, "dev.output")),2)
xmatch=round(np.mean(accs)*100,4)

In [114]:
print(f"BLEU: {dev_bleu}, X-Match: {xmatch}")

BLEU: 74.33, X-Match: 58.0


In [115]:
print(p[1])

public override string ToString(){return pattern();}


In [22]:
# import wandb
# wandb.login()

In [120]:
# login huggerface
# !huggingface-cli login

In [ ]:
# repo_id = "judynguyen16/graphcodebert-code-translation-java-cs"
# !huggingface-cli repo create "judynguyen16/graphcodebert-code-translation-java-cs"

git version 2.25.1
Looks like you do not have git-lfs installed, please install. You can install from https://git-lfs.github.com/. Then run `git lfs install` (you only have to do this once).

You are about to create judynguyen16/judynguyen16/graphcodebert--code-translation-java-cs
Proceed? [Y/n] ^C
Traceback (most recent call last):
  File "/opt/conda/envs/pytorch/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/commands/huggingface_cli.py", line 57, in main
    service.run()
  File "/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/commands/user.py", line 285, in run
    choice = input("Proceed? [Y/n] ").lower()
KeyboardInterrupt


In [ ]:
from transformers import AutoModel, AutoTokenizer
# Push model and tokenizer to Hugging Face Hub
from huggingface_hub import upload_file

repo_id = "judynguyen16/graphcodebert--code-translation-java-cs"

files_to_upload = {
    "pytorch_model.bin": "/home/ubuntu/judy/transformer-code-translation/GraphCodeBERT/translation/saved_models/java-cs/checkpoint-best-ppl/pytorch_model.bin",
}

for dest_path, local_path in files_to_upload.items():
    upload_file(
        path_or_fileobj=local_path,
        path_in_repo=dest_path,
        repo_id=repo_id,
        repo_type="model",
    )


HFValidationError: Cannot have -- or .. in repo_id: 'judynguyen16/graphcodebert--code-translation-java-cs'.